### This code extracts documents and related comments from documents in different categories. There are 10 categories in regulation.gov

In [1]:
from pickle import dump, load
import pandas
import requests
import urllib.request
from bs4 import BeautifulSoup
import PyPDF2
api_key = 'vT4R3vZ8RpZhnCpgeCPx1LdWRSZS8yxHHGquPrxm'

In [2]:
# def read_file_get_docid(filepath):
#     dump_df = load(open(filepath,'rb'))
#     df_with_comments = dump_df[dump_df.numberOfCommentsReceived > 0]
#     df_with_comments =df_with_comments.sort(['numberOfCommentsReceived'], ascending=[False])
#     doc_id = df_with_comments.documentId
#     doc_type = df_with_comments.documentType
#     #return [doc_id,set(doc_type)]
#     return df_with_comments

In [16]:
# [doc_id_list,types] = read_file_get_docid('data/AD_doc_list') #choose the category dump file
# print(types)
# # document ID with 4 parts represent documents. 3 parts represent dockets 
# doc_ids = [doc_id for doc_id in doc_id_list if len(doc_id.split('-')) == 4]
# len(doc_ids)

{'Proposed Rule', 'Rule', 'Other', 'Notice'}


/Users/kinshuk/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


4421

### Using regulations.gov API
* We need to use the API to retrieve each document content. This API will use document_id that we extracted from the file above.
* For each document_id, we will need to construct comment_id based on the total number of comments on it.

In [2]:
def download_file(download_url):
    try:
        response = urllib.request.urlopen(download_url)
        file = open("document.pdf", 'wb')
        file.write(response.read())
        file.close()
    except:
        print("(downloading the pdf exception)error log" +  download_url)
    
def get_attached_comments(comment_id, key=api_key):
    #print(each_id) # fro debugging
    #open the api to get file url
    url = "http://api.data.gov:80/regulations/v3/document.json?api_key="+key+"&documentId="+comment_id
    try:
        response = requests.get(url)
    except:
        print("(api opening of attached comment exception)error log" +  url)
    if response.status_code != 200:
        print("status code " +str(response.status_code)+" (get_attached_comments) program will break at this point which is ok because we dont need inconsistent data. Run again ")
    data = response.json()
    if "fileFormats" in data["attachments"][0]:
        att_count = len(data["attachments"][0]["fileFormats"])
    else:
        att_count = len(data["attachments"][1]["fileFormats"])
    comment_text =""
    for i in range(att_count):
        if data["attachments"][0]["fileFormats"][i].endswith("pdf"):
            link = data["attachments"][0]["fileFormats"][i] 
            access_link = link+'&api_key='+key
            #download file(pdf) and read pdf (page by page)
            download_file(access_link)
            pdfFileObj = open('document.pdf','rb')     #'rb' for read binary mode
            try:
                pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
                pno = pdfReader.numPages
                for i in range(pno):
                    pageObj = pdfReader.getPage(i)          #'i' is the page number
                    comment_text += pageObj.extractText()
            except:
                print("(pdf exception)cant read "+comment_id ) # prints in case we are not able to read file
            break # execute the whole thing for 1st found pdf
    return comment_text

In [3]:
def get_document_comments_from_api(docketId,key=api_key):
    offset=0
    url = "http://api.data.gov:80/regulations/v3/documents.json?api_key="+key+"&countsOnly=1&dct=PS&dktid="+docketId
    try:
        response = requests.get(url)
    except:
        print("(api opening comment count)error log"+url) # prints in case we are not able to read file
    if response.status_code != 200:
        print("status code"+str(response.status_code) + " (get_document_comments_from_api) program will break at this point which is ok because we dont need inconsistent data. Run again")
    data = response.json()
    total = data['totalNumRecords']
    com_list =[]
    for i in range(0,total,1000):
        url = "http://api.data.gov:80/regulations/v3/documents.json?api_key="+key+"&countsOnly=0&&rpp=1000&po="+str(i)+"&dct=PS&dktid="+docketId
        try:
            response = requests.get(url)
        except:
            print("(api opening actual comments)error log"+url) # prints in case we are not able to read file
        #print("Offset:"+str(i)+" Code:"+str(response.status_code))
        if response.status_code != 200:
            print(response.status_code)
        data = response.json()
        com_list += data['documents']
    com_df = pandas.DataFrame(com_list)
    return com_df

def get_document_content_from_api(docId,key=api_key):
    url = "http://api.data.gov:80/regulations/v3/document.json?api_key="+key+"&documentId="+docId
    try:
        response = requests.get(url)
    except:
         print("(api opening doc exception) error log"+url)   
    if response.status_code != 200:
        print("status code "+str(response.status_code)+" (get_document_content_from_api) program will break at this point which is ok because we dont need inconsistent data. Run again")
    data = response.json()
    
    # Get HTML for document content
    if(len(data['fileFormats']) == 2):    
        link = data['fileFormats'][1] # The second link is the document in HTML format
    else:
        link = data['fileFormats'][0]
    access_link = link+'&api_key='+key
    
    try:
        with urllib.request.urlopen(access_link) as response:
            html = response.read()
    except:
        print("doc file opening exception")
    
    # We are interested in the pre tag of the HTML content
    soup = BeautifulSoup(html, "lxml")
    content = soup.find_all('pre')
    
    # Now we need to construct comment_id from document_id
    docket_id = '-'.join(docId.split('-')[:3])
    comment_df = get_document_comments_from_api(docket_id)
    # get comment text where exists
    comment_list =[]
    if not comment_df.empty:
        if "commentText" in comment_df:
            comment_text =comment_df[comment_df.commentText.notnull()].commentText
            comment_list =comment_text.tolist()
        #get doc id where there is attchment
        c_ids = comment_df[comment_df.attachmentCount>0].documentId
        # get comment for each id in list
        for each_id in c_ids.unique():
            comment_list.append(get_attached_comments(each_id))
    doc_dict = {
        "text":content,
        "comment_list":comment_list
    }
    return doc_dict

#### Running it on one document

In [4]:
# data format is array of dicts
def get_one_doc(name,docid):
    #open file - get present data (empty to begin witj)
    filepath = "data/"+name+"_doc_content"
    inp =open(filepath,'rb')
    doc_collection = load(inp)
    inp.close()
    #get that one doc
    resp = get_document_content_from_api(docid)
    doc_collection.append(resp)
    #put back in file 
    output = open(filepath, 'wb')
    dump(doc_collection, output, -1)
    output.close()


In [6]:
# run this line once every 1 hr
get_one_doc("Kinshuk","ATBCB-2010-0001-0001")

(pdf exception)cant read ATBCB-2010-0001-0090


# BELOW THIS IS TESTING, NOT NEEDED TO GET DOCUMENT AND COMMENTS CURRENTLY

In [85]:
df_test =read_file_get_docid('data/PRE_doc_list')
df_test.reset_index(inplace = True)

In [90]:
df_test["numberOfCommentsReceived"][20:65] #60,61,62,63

20    4630
21    3136
22    3123
23    3032
24    2441
25    2188
26    2076
27    1870
28    1684
29    1633
30    1488
31    1412
32    1333
33    1328
34    1228
35    1100
36    1085
37    1071
38    1025
39     853
40     824
41     736
42     713
43     690
44     668
45     649
46     575
47     554
48     536
49     531
50     524
51     516
52     492
53     465
54     450
55     448
56     435
57     433
58     400
59     399
60     395
61     378
62     377
63     370
64     366
Name: numberOfCommentsReceived, dtype: int64

In [89]:
df_test["documentId"][60:65]

60       BOR-2008-0004-0001
61       HUD-2011-0014-0001
62       HUD-2006-0204-0001
63    HHS-OS-2015-0005-0001
64       BIA-2013-0007-0087
Name: documentId, dtype: object